# Illegal Move Detection and Game Feedback System

In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
import chess
import chess.pgn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import chess.pgn
import pandas as pd

# Initialize a list to store data
data = []

# Path to your PGN file
pgn_file = r"C:\Users\la7tim\Downloads\lichess_db_standard_rated_2014-09.pgn\lichess_db_standard_rated_2014-09.pgn"

# Open the PGN file
with open(pgn_file) as pgn:
    while True:
        game = chess.pgn.read_game(pgn)
        if game is None:
            break

        # Extract game metadata
        game_result = game.headers.get("Result")
        white_elo = game.headers.get("WhiteElo")
        black_elo = game.headers.get("BlackElo")

        # Iterate through moves
        board = game.board()
        for move in game.mainline_moves():
            # Get FEN before the move
            fen = board.fen()
            board.push(move)  # Apply the move

            # Store FEN, move, and metadata
            data.append({
                "FEN": fen,
                "Move": move.uci(),
                "Result": game_result,
                "WhiteElo": white_elo,
                "BlackElo": black_elo
            })

# Create a DataFrame
df = pd.DataFrame(data)
print(df.head())

# Save to CSV
df.to_csv("chess_dataset.csv", index=False)


                                                 FEN  Move Result WhiteElo  \
0  rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...  e2e4    0-1     1833   
1  rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...  d7d5    0-1     1833   
2  rnbqkbnr/ppp1pppp/8/3p4/4P3/8/PPPP1PPP/RNBQKBN...  g1f3    0-1     1833   
3  rnbqkbnr/ppp1pppp/8/3p4/4P3/5N2/PPPP1PPP/RNBQK...  d5e4    0-1     1833   
4  rnbqkbnr/ppp1pppp/8/8/4p3/5N2/PPPP1PPP/RNBQKB1...  f3e5    0-1     1833   

  BlackElo  
0     1823  
1     1823  
2     1823  
3     1823  
4     1823  


In [ ]:
# Step 2: Preprocessing
# Encode categorical variables (e.g., moves)
label_encoder = LabelEncoder()
df['Move_encoded'] = label_encoder.fit_transform(df['Move'])

# Select features and target
X = df[['Move_encoded', 'WhiteElo', 'BlackElo']]
y = df['Result']  # Target column

In [ ]:
# Step 3: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Training and Testing Sets Prepared")

In [ ]:
# Step 4: Model Training
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
print("Model Training Completed")

In [ ]:
# Step 5: Evaluation
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Step 6: Feature Importance
importances = model.feature_importances_
features = X.columns
plt.figure(figsize=(8, 6))
sns.barplot(x=importances, y=features)
plt.title("Feature Importance")
plt.show()

In [ ]:
# Step 7: Illegal Move Detection
# Function to validate moves using python-chess
def detect_illegal_moves(fen, move):
    board = chess.Board(fen)
    try:
        board.push_uci(move)
        return "Legal"
    except ValueError:
        return "Illegal"

# Apply detection on a sample of the dataset
data['Move_Legality'] = data.apply(lambda row: detect_illegal_moves(row['FEN'], row['Move']), axis=1)
print("Illegal Move Detection Completed")
print(data[['FEN', 'Move', 'Move_Legality']].head())

In [ ]:
# Step 8: Game Feedback System
# Analyze move sequences and provide insights
def generate_feedback(fen, move):
    board = chess.Board(fen)
    if detect_illegal_moves(fen, move) == "Illegal":
        return "This move is illegal! Please check the rules."
    else:
        return "This move is valid. Keep up the good strategy!"

# Apply feedback generation to the dataset
data['Feedback'] = data.apply(lambda row: generate_feedback(row['FEN'], row['Move']), axis=1)
print("Game Feedback Generated")
print(data[['FEN', 'Move', 'Feedback']].head())

In [ ]:
# Step 9: Save Processed Data
data.to_csv('processed_chess_data.csv', index=False)
print("Processed Data Saved")